## Imports

In [1]:
import sys
import os
import importlib
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro, kstest
from scipy import stats  

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet, RidgeCV, LassoCV,ElasticNetCV, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score, classification_report, f1_score, matthews_corrcoef, mean_squared_error,r2_score, roc_auc_score, roc_curve, auc, confusion_matrix, log_loss
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import kstest_normal
from timeit import default_timer as timer
from tqdm import tqdm  
from typing import Optional
from joblib import Parallel, delayed
import pickle

module_path = str(Path("../src/data").resolve())
if module_path not in sys.path:
    sys.path.append(module_path)

import LogRegFxF as LR
import preprocessing as prep

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

In [2]:
sys.path

['/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python313.zip',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/lib-dynload',
 '',
 '/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages',
 '/home/lestrada/tumor_type_prediction/src/data']

In [30]:
import LogRegFxF as LR
import preprocessing as prep
import feature_selection as fs

In [43]:
importlib.reload(fs)

dir(fs)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'binary_labeling',
 'np',
 'pd']

## Data Import

In [5]:
#Proteins quantification intensities file
processed_data = '2024.10.23_CJ_pancancer_250/'
folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
PREPROCESSED_FP_INTENSITY = 'preprocessed_fp_with_ref.csv'
intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
input_quantifications = pd.read_csv(intensity_path_file)

#--------------------------------------------------------------------------------

#Samples metadata (oncotree classification) file.
metadata_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_MTBs_Evaluation/'
metadata_file = 'METADATA_PAN_CANCER_Batch300.xlsx'
the_metadata_file = metadata_path + metadata_file
input_metadata = pd.read_excel(the_metadata_file)

#--------------------------------------------------------------------------------

# # Proteins quantification z-scores file
# processed_data = '2024.10.23_CJ_pancancer_250/'
# folder_path = '/media/kusterlab/internal_projects/active/TOPAS/WP31/Playground/Retrospective_study/'
# PREPROCESSED_FP_INTENSITY = 'full_proteome_measures_z.tsv'
# intensity_path_file = folder_path + processed_data + PREPROCESSED_FP_INTENSITY
# df_Z_scores = pd.read_csv(intensity_path_file, sep='\t')

## Data Preprocessing

In [ ]:
#Peptides quantification intensities post-processing

# Protein quantification intensities post-processing
input_quantifications = input_quantifications.set_index(input_quantifications.columns[0])
peptides_quant_info = prep.post_process_meta_intensities(input_quantifications.iloc[:,int(input_quantifications.shape[1]/2):].T ) #clean dataframe from regex characers
proteins_quant = input_quantifications.iloc[:,:int(input_quantifications.shape[1]/2)].T #subset protein measurements from dataset

#Imputation
prot_quant_imputed = prep.impute_normal_down_shift_distribution(proteins_quant) #Imputation of missing values in protein intensities using normal distribution down-shift method
na_columns = prot_quant_imputed.isna().any()
na_columns_true = na_columns[na_columns].index.tolist()
print("Proteins with  empty values:", na_columns_true)

#Cleaning sample names
prot_quant_imputed.reset_index(inplace=True)
prot_quant_imputed.rename(columns={'index': 'Sample name'}, inplace=True)
prot_quant_imputed['Sample name'] = prot_quant_imputed['Sample name'].str.replace('pat_', '')

#Dataset with protein intensities and metadata
samples_metadata = input_metadata[["Sample name", "code_oncotree",]] #sample metadata e.g. class, TCC, tissue of origin, etc.
initial_df = samples_metadata.merge(prot_quant_imputed, left_on='Sample name', right_on='Sample name')

#Peptides quantification to binary dataset
peptides_df_binary = pd.DataFrame(
    np.where(peptides_quant_info > 1, 1, 0), #if the # of peptides > 1, then turns to 1, otherwise 0. 
    index=peptides_quant_info.index,
    columns=peptides_quant_info.columns  
)
peptides_df_binary.reset_index(inplace=True) #Moves the index to a column. Allows to obtain patient id
peptides_df_binary.replace('Identification metadata ','',regex=True, inplace=True) #Removes text from id's
peptides_df_binary = samples_metadata.merge(peptides_df_binary, left_on='Sample name', right_on='index') #merging both data sets by Sample Name, ontaining a dataset with sample, classification and peptide binary count
peptides_df_binary.drop('index', axis=1, inplace=True)

peptides_df_binary

(2135, 13017)


/home/lestrada/miniconda3/envs/tumor_type_clasifier/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:2015: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/lestrada/tumor_type_prediction/src/data/preprocessing.py:50: RuntimeWarning: Mean of empty slice
  temp_mean = np.nanmean(temp)


Proteins with  empty values: ['PTGER4', 'CD19', 'FOXO4', 'CRYGA', 'HNRNPCL3;HNRNPCL4', 'MYBPHL']


## Data Set Split

In [7]:
#Removing samples not part of the Oncotree classification
NOS_cases = ['CUPNOS', 'ADNOS', 'SARCNOS', 'SCCNOS', 'missing', 'SOLIDNOS', 'RCSNOS', 'GCTNOS']
ml_initial_df = prep.remove_class(initial_df, NOS_cases, 'code_oncotree')

Removed samples: 191
Remaining samples: 1469


In [8]:
# Splitting dataset into training and held-out sets1
training_df, held_out_df = prep.data_split(ml_initial_df, split_size=0.25, classified_by='code_oncotree')

Classes with only one sample: 70
Training set samples: 1119
Held-out set samples: 350


In [44]:
ARMS_training_df = fs.binary_labeling(training_df, classified_by='code_oncotree', true_class=['ARMS'])
ARMS_ho_df = fs.binary_labeling(held_out_df, classified_by='code_oncotree', true_class=['ARMS'])


Number of samples per class:
Classifier
0    1076
1      43
Name: count, dtype: int64


Number of samples per class:
Classifier
0    336
1     14
Name: count, dtype: int64



## Feature Selection

In [ ]:
#cross validation for final model
def nested_cross_validation_logistic_regression(train_df:pd.DataFrame,classified_by:str, random_state=93):
    """
    classification_criteria: insert the 
    """
    y = train_df['Classifier']  # True values (dependent variable)
    X = train_df.drop(columns=['Sample name', classified_by, 'Classifier'], inplace=True)  # Independent variables (proteins)

    # Define the hyperparameter grid for Logistic Regression
    param_grid = {'C': [0.1, 1, 10]}

    # Set up the Logistic Regression model with L2 regularization (Ridge)
    logreg = LogisticRegression(penalty='elasticnet',
                                solver='saga',
                                l1_ratio=0,  # Ridge regularization
                                max_iter=10000,
                                class_weight='balanced',
                                warm_start=True)

    # Set up MCC scorer
    mcc_scorer = make_scorer(matthews_corrcoef)

    # Set up inner and outer cross-validation
    inner_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)
    outer_cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=random_state)

    # Lists to store results
    outer_scores = []
    best_params = []
    inner_fold_cycle = 1
    # Perform nested cross-validation
    for train_idx, test_idx in outer_cv.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # Inner cross-validation with GridSearchCV
        grid_search = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=inner_cv, scoring=mcc_scorer)
        grid_search.fit(X_train, y_train)

        # Get the best parameters and the score for the inner CV
        best_param = grid_search.best_params_
        best_score = grid_search.best_score_

        # Evaluate the best model on the outer test set
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)
        outer_score = matthews_corrcoef(y_test, y_pred)

        # Store the results
        outer_scores.append(outer_score)
        best_params.append(best_param)

        # Print the best parameters and the cross-validation score for each fold
        print(f"{inner_fold_cycle} Inner fold best parameter={best_param}, Score={best_score:.4f}, Outer MCC Score(held-out): {outer_score:.4f}")
        inner_fold_cycle += 1
 
    # Print overall mean MCC score
    print(f"Average MCC across all outer folds: {np.mean(outer_scores):.4f}")
    
    return outer_scores, best_params
